In [1]:
import os
import IPython
import matplotlib
import matplotlib.pyplot as plt
import requests
import torch
import transformers

In [2]:
from transformers import BertForQuestionAnswering, AutoTokenizer, DefaultDataCollator, TrainingArguments, Trainer
import torch

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = BertForQuestionAnswering.from_pretrained(model_name)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
context = "The University of California was founded in 1868, located in Berkeley."
question = "When was the University of California established?"

In [12]:
inputs = tokenizer(question, context, return_tensors='pt')
with torch.no_grad():
    outputs = model(**inputs)

# Find the tokens with the highest `start` and `end` scores
answer_start = torch.argmax(outputs.start_logits)
answer_end = torch.argmax(outputs.end_logits) + 1

# Convert tokens to answer string
answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs.input_ids[0, answer_start:answer_end]))
print("Answer:", answer)

Answer: ? [SEP] the university of california was founded in 1868, located in berkeley


In [13]:
print(inputs)

{'input_ids': tensor([[ 101, 2043, 2001, 1996, 2118, 1997, 2662, 2511, 1029,  102, 1996, 2118,
         1997, 2662, 2001, 2631, 1999, 7582, 1010, 2284, 1999, 8256, 1012,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [16]:
print(outputs.keys())
print(outputs)

odict_keys(['start_logits', 'end_logits'])
QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[0.0861, 0.3946, 0.5680, 0.6106, 0.4046, 0.3115, 0.3395, 0.5060, 1.1208,
         0.0550, 0.6062, 0.5184, 0.3386, 0.5573, 0.6790, 0.4443, 0.2488, 0.4463,
         0.8225, 0.6303, 0.3360, 0.7104, 0.5018, 0.4873]]), end_logits=tensor([[-0.0648,  0.1363,  0.0164,  0.2471,  0.1026,  0.0462,  0.4932,  0.1213,
         -0.0507, -0.0425,  0.3365,  0.3966,  0.2094,  0.7707,  0.3421,  0.4063,
          0.3470,  0.0225,  0.0718,  0.4710,  0.3110,  1.0933,  0.2035,  0.1882]]), hidden_states=None, attentions=None)
